In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import libraries

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import numpy as np
import pandas as pd

Read all the files and create a unified dataset about the pression for every points and for every seconds: on the rows, there are the 11160 points, while, on the columns, the pressure for the 10010 seconds (taken at interval of 10s).

In [ ]:
# insert your path
path='./drive/MyDrive/Colab_Notebooks/Datamining_Project/'

In [ ]:
li=[]
for i in range(1001):
    df = pd.read_csv(path + 'data/dataFile0.'+str(i)+'.csv', index_col=None, header=0)
    df.set_index(keys=['Points:0','Points:1','Points:2'], inplace=True)
    li.append(df['p'].to_frame())
    if i==0:
        li[0].rename(columns={'p':' '.join(['p',str(0)])}, inplace=True)
        tot=li[0]
    else:
        li[i].rename(columns={'p':' '.join(['p',str(i)])}, inplace=True)
        tot = tot.join(li[i])

In [ ]:
# only to save the file
# tot.to_csv(path+"tot_pressure.csv", index=[['Points:0','Points:1','Points:2']])

In [ ]:
# only to read data
tot = pd.read_csv(path+'tot_pressure.csv',index_col=['Points:0','Points:1','Points:2'])
tot_df = tot.T
tot = tot_df.to_numpy()

Check if there are any missing values.

In [ ]:
tot_df.T[tot_df.isna().any()]

,,,p 0,p 1,p 2,p 3,p 4,p 5,p 6,p 7,p 8,p 9,p 10,p 11,p 12,p 13,p 14,p 15,p 16,p 17,p 18,p 19,p 20,p 21,p 22,p 23,p 24,p 25,p 26,p 27,p 28,p 29,p 30,p 31,p 32,p 33,p 34,p 35,p 36,p 37,p 38,p 39,...,p 961,p 962,p 963,p 964,p 965,p 966,p 967,p 968,p 969,p 970,p 971,p 972,p 973,p 974,p 975,p 976,p 977,p 978,p 979,p 980,p 981,p 982,p 983,p 984,p 985,p 986,p 987,p 988,p 989,p 990,p 991,p 992,p 993,p 994,p 995,p 996,p 997,p 998,p 999,p 1000
Points:0,Points:1,Points:2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


Summary statistics of the dataset

In [ ]:
desc=tot_df.describe()

In [ ]:
desc

Points:0      -24.750               ...        64.694        65.000
Points:1     -9.3333      -10.0000  ...       10.0000       10.0000
Points:2            0            0  ...             0             0
count     1001.000000  1001.000000  ...  1.001000e+03  1.001000e+03
mean         0.000319     0.000320  ...  6.639557e-07  2.779778e-07
std          0.003166     0.003165  ...  1.386600e-05  8.413391e-06
min          0.000000     0.000000  ... -2.264600e-05 -1.708800e-05
25%          0.000148     0.000149  ... -5.422400e-06 -3.993900e-06
50%          0.000184     0.000185  ... -2.284400e-06 -2.053200e-06
75%          0.000226     0.000227  ...  5.281100e-07  2.509700e-07
max          0.100190     0.100170  ...  2.438400e-04  5.675000e-05

[8 rows x 11160 columns]

In [ ]:
max(desc.T['max'])

0.10075

In [ ]:
min(desc.T['min'])

-0.0035479999999999

Define the function to prepare data for the LSTM model.

In [ ]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, Y = list(), list()
    for i in range(len(sequences)+1):
        # find the end of this pattern
        end_ix = i + n_steps
        
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
		
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        Y.append(seq_y)
        
    return np.array(X), np.array(Y) # the outputs of the function are, respectively, the input data and the target data of the model

Splitting data into training set - test set

In [ ]:
# Training set - Test set
# 80% - 20%, choosed chronologically

tot_train = tot[:800,:]
tot_test = tot[800:,:]

Build the model

In [ ]:
# choose a number of time steps (trade off computational between cost and quality of the model)
n_steps = 75
# convert into input/output
X_train, Y_train = split_sequences(tot_train, n_steps)
n_features = X_train.shape[2]
# define model
model = Sequential()
model.add(LSTM(100, activation='tanh', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation='tanh'))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X_train, Y_train, epochs=400, verbose=0)
# convert into input/output
X_test, Y_test = split_sequences(tot_test, n_steps)

nan


Finally compute the prediction

In [ ]:
Y_pred = model.predict(X_test, verbose=0)

Save data

In [ ]:
# only to save data
# pd.DataFrame(Y_pred).to_csv(path + "Y_pres_lstm.csv", index=False)

AttributeError: ignored

In [ ]:
# only to read data
# Y_pred = pd.read_csv(path+'Y_pres_lstm.csv')

Create 126 csv files with the predictions.

In [ ]:
# to save results
Y_pred = Y_pred.to_numpy()
data_pred=tot_df.T.reset_index()[['Points:0','Points:1','Points:2']]

In [ ]:
# save data: a dataset is created for each second and it contains the prediction for every point
for i in range(len(Y_pred)):
  j=i+n_steps
  if i==0:
    data_pred.insert(0,'Predicted p',Y_pred[i])
    data_pred.to_csv(path+"dataLSTM/dataPred_"+str(1000-200+j)+".csv", index=False)
  if i!=0:
    data_pred['Predicted p']=Y_pred[i]
    data_pred.to_csv(path+"dataLSTM/dataPred_"+str(1000-200+j)+".csv", index=False)